# Data Cleaning-Colab1
link to colab
https://colab.research.google.com/drive/1gqyuCIic-wTC2KnmkEdAQ7rURqj6ztHv


## Presentation
The purpose of this notebook, is to get an overview of the data, and to make it ready and usable for true exploration and data manipulation. To do so, it will be a mix of studying the data, and cleaning it. As a rule, columns with almost no actual values, will be removed (full of NaN). Also, some lines of code, have been rendered inactive using the # sign. this is meerely done, so that the colab runs a bit more smoorth, but they where all part of the discovery process. given that the data originally contained approx 180.000 observations. is was determained that there is some place, to drop some where needed. 

Firstly, we will have to get the data into the colab from kaggle. We opted to do this, by putting the data into a dropbox as a zip file (due to its size), and import it from there. 

In [0]:
!wget https://www.dropbox.com/s/s8nl55go4o8powf/globalterrorismdb_0718dist.csv.zip
#Unzipping in order to get it to read as a csv
!unzip globalterrorismdb_0718dist.csv.zip

--2019-09-26 12:03:26--  https://www.dropbox.com/s/s8nl55go4o8powf/globalterrorismdb_0718dist.csv.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/s8nl55go4o8powf/globalterrorismdb_0718dist.csv.zip [following]
--2019-09-26 12:03:27--  https://www.dropbox.com/s/raw/s8nl55go4o8powf/globalterrorismdb_0718dist.csv.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2b628c166050f00a364b415520.dl.dropboxusercontent.com/cd/0/inline/ApSKXoYFjcbmS_H7rSi8plA6Y05mKDJlMh3_cp4lVUELArOvni1cAI5CYrEa5LIL8hkKdln2kc53Pk_cPcSpBA8HCRhSHKRhJfBTB0Yo7g5DVv-nqa_EEOqo6FYjBR3QuCI/file# [following]
--2019-09-26 12:03:27--  https://uc2b628c166050f00a364b415520.dl.dropboxusercontent.com/cd/0/inline/ApSKXoYFjcbmS_H7rSi8plA6Y05mKDJlMh3_cp4lVUELA

In [0]:
import numpy as np
import pandas as pd
full_data=pd.read_csv('globalterrorismdb_0718dist.csv', encoding="ISO-8859-1")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#full_data.head(2)

In [0]:
full_data.info(verbose=True, null_counts=True)
#In order to get the correct format on the .info function, we had to use the additional arguments. othervise the number of actual values in a column wouldn't show

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Data columns (total 135 columns):
eventid               181691 non-null int64
iyear                 181691 non-null int64
imonth                181691 non-null int64
iday                  181691 non-null int64
approxdate            9239 non-null object
extended              181691 non-null int64
resolution            2220 non-null object
country               181691 non-null int64
country_txt           181691 non-null object
region                181691 non-null int64
region_txt            181691 non-null object
provstate             181270 non-null object
city                  181257 non-null object
latitude              177135 non-null float64
longitude             177134 non-null float64
specificity           181685 non-null float64
vicinity              181691 non-null int64
location              55495 non-null object
summary               115562 non-null object
crit1                 181691 non-null int64

# Acutual cleaning

Based on the overview and info, gotten from the previous chapter, together with the definitions given in the data description, from the Kaggle creator(See the stakeholder report for more info). we decided, to augment the data to only contain the following columns.

after picking the columns these get put in to a new data frame, in order to have the original data ready at hand if needed

In [0]:
df=full_data.loc[:,('iyear','imonth','iday','extended','resolution','country_txt', 'region_txt','latitude','longitude', 'success', 'nkill', 'nkillter', 'nwound','property', 'vicinity', 'suicide', 'claimed','gname', 'individual', 'crit1','crit2',	'crit3','attacktype1_txt','attacktype2_txt','attacktype3_txt','targtype1_txt','targtype2_txt','targtype3_txt','weaptype1_txt','weaptype2_txt','weaptype3_txt')]
#df.head()

## Timestamp
The first thing we decided, was important, was to get the three different columns iyear, imonth and iday. strung together into an actual datetime format that can be easier to read and use in the future. 

In [0]:
#df.imonth.value_counts()
# 20 observations does not have month (month=0)
# 891 observations does not have day (day=0)

In [0]:
#In order to get only usable data, the 0's where replaced with nan values and dropped. 
df['imonth']=df['imonth'].replace(0, np.nan)
df['iday']=df['iday'].replace(0, np.nan)
df.dropna(subset=['imonth', 'iday'],inplace=True)

In [0]:
date = pd.DataFrame({'year': df['iyear'], 'month':df['imonth'],'day': df['iday']})
df.insert(0,'timestamp',pd.to_datetime(date))

In [0]:
#After having converted the columns in to one datetime, we dropped the original in order, to not have redundant information
df.drop(columns=['iyear','imonth','iday'],inplace=True)

## NaN and -9 (unknown)
Alot of the data in the CSV file. cointains data to varying degree, that are either NaN or -9/-99 the -9/-99 format is the publishers doing, for the places where they do not know or are uncertain. 

We how ever wanted to get rid of those, in order to get a working data frame for later use. 

This is done by, turning the -9/-99 data into NaN and dropping it 

In [0]:
#print(df.nkill.isnull().sum()) 10313
#drop na in nkill
df.dropna(subset=['nkill'],inplace=True)

In [0]:
#print(df.nwound.isnull().sum()) 6561
#drop na in nwound
df.dropna(subset=['nwound'],inplace=True)

In [0]:
#print(df.property.value_counts())
df['property']=df['property'].replace(-9, np.nan)
#print(df.property.isnull().sum()) # 19248
df.dropna(subset=['property'],inplace=True)

In [0]:
#df.claimed.value_counts()
df['claimed']=df['claimed'].replace(-9, np.nan)
#print(df.claimed.isnull().sum()) 58458
# 66581 NaN   - we set these to 0
df['claimed']=df['claimed'].replace(np.nan, 0)

In [0]:
#print(df.vicinity.value_counts())
df['vicinity']=df['vicinity'].replace(-9, np.nan)
#print(df.vicinity.isnull().sum()) # 23
df.dropna(subset=['vicinity'],inplace=True)

In [0]:
#Replace nkiltter nan with 0
df['nkillter'].fillna(0, inplace=True)

In [0]:
df.dropna(subset=['latitude','longitude'],inplace=True)

## Dummies

Some of the original data, was arranged in such a way, that it became unwieldy due to it being categorical data filling the same column in order to get around this, we opted to make them in to dummy variables, so only one thing, pr column could be true (1,0). this was done for all the different attack types, targettypes and weapontypes

### attacktype 1-3

In [0]:
#First we look in the to different types, to see what simiarlarities that are between them
#print(df.attacktype1_txt.unique())
#print(df.attacktype2_txt.unique())
#print(df.attacktype3_txt.unique())

In [0]:
#here we create a different dateframe containing the columns now turned into dummies
df_dum=pd.get_dummies(df,columns=['attacktype1_txt','attacktype2_txt','attacktype3_txt'])

In [0]:
#now in order to get a good overview of the data, and to reduce some of the dummies. All the diffrent types of f.x Armed assault
#Gets melted together with eachother, and turned in to one column called Armed Assault. 

df['Armed_Assault']=(df_dum['attacktype1_txt_Armed Assault']+df_dum['attacktype2_txt_Armed Assault']+df_dum['attacktype3_txt_Armed Assault']).replace(2, 1).replace(3,1)
df['Bombing_Explosion']=(df_dum['attacktype1_txt_Bombing/Explosion']+df_dum['attacktype2_txt_Bombing/Explosion']+df_dum['attacktype3_txt_Bombing/Explosion']).replace(2, 1).replace(3,1)
df['Facility_Infrastructure_Attack']=(df_dum['attacktype1_txt_Facility/Infrastructure Attack']+df_dum['attacktype2_txt_Facility/Infrastructure Attack']+df_dum['attacktype3_txt_Facility/Infrastructure Attack']).replace(2, 1).replace(3,1)
df['Hostage_Taking_Kidnapping']=(df_dum['attacktype1_txt_Hostage Taking (Kidnapping)']+df_dum['attacktype2_txt_Hostage Taking (Kidnapping)']+df_dum['attacktype3_txt_Hostage Taking (Kidnapping)']).replace(2, 1).replace(3,1)
df['Unarmed_Assault']=(df_dum['attacktype1_txt_Unarmed Assault']+df_dum['attacktype2_txt_Unarmed Assault']+df_dum['attacktype3_txt_Unarmed Assault']).replace(2, 1).replace(3,1)
df['Hostage_Taking_Barricade_Incident']=(df_dum['attacktype1_txt_Hostage Taking (Barricade Incident)']+df_dum['attacktype2_txt_Hostage Taking (Barricade Incident)']+df_dum['attacktype3_txt_Hostage Taking (Barricade Incident)']).replace(2, 1).replace(3,1)
df['Assassination']=(df_dum.attacktype1_txt_Assassination+df_dum.attacktype2_txt_Assassination+df_dum.attacktype3_txt_Assassination).replace(2, 1).replace(3,1)
df['Hijacking']=(df_dum.attacktype1_txt_Hijacking+df_dum.attacktype2_txt_Hijacking+df_dum.attacktype3_txt_Hijacking).replace(2, 1).replace(3,1)
df['Unknown']=(df_dum.attacktype1_txt_Unknown+df_dum.attacktype2_txt_Unknown).replace(2, 1)

In [0]:
#since we now have the data we need, in other columns we drop the old ones. 
df.drop(columns=['attacktype1_txt', 'attacktype2_txt','attacktype3_txt'],inplace=True)

### targettype 1-3

The Method from attack type gets replicated here, and will also be replicated for weapon types. 

In [0]:
#print(df.targtype1_txt.unique())
#print(df.targtype2_txt.unique())
#print(df.targtype3_txt.unique())

In [0]:
df_dum=pd.get_dummies(df,columns=['targtype1_txt','targtype2_txt','targtype3_txt'])

In [0]:
df['Abortion_Related']=(df_dum['targtype1_txt_Abortion Related']+df_dum['targtype2_txt_Abortion Related']).replace(2, 1).replace(3,1)
df['Airports_Aircraft']=(df_dum['targtype1_txt_Airports & Aircraft']+df_dum['targtype2_txt_Airports & Aircraft']+df_dum['targtype3_txt_Airports & Aircraft']).replace(2, 1).replace(3,1)
df['Business']=(df_dum.targtype1_txt_Business+df_dum.targtype2_txt_Business+df_dum.targtype3_txt_Business).replace(2, 1).replace(3,1)
df['Educational_Institution']=(df_dum['targtype1_txt_Educational Institution']+df_dum['targtype2_txt_Educational Institution']+df_dum['targtype3_txt_Educational Institution']).replace(2, 1).replace(3,1)
df['Food_Water_Supply']=(df_dum['targtype1_txt_Food or Water Supply']+df_dum['targtype2_txt_Food or Water Supply']).replace(2, 1)
df['Government_Diplomatic']=(df_dum['targtype1_txt_Government (Diplomatic)']+df_dum['targtype2_txt_Government (Diplomatic)']+df_dum['targtype3_txt_Government (Diplomatic)']).replace(2, 1).replace(3,1)
df['Government_General']=(df_dum['targtype1_txt_Government (General)']+df_dum['targtype2_txt_Government (General)']+df_dum['targtype3_txt_Government (General)']).replace(2, 1).replace(3,1)
df['Journalists_Media']=(df_dum['targtype1_txt_Journalists & Media']+df_dum['targtype2_txt_Journalists & Media']+df_dum['targtype3_txt_Journalists & Media']).replace(2, 1).replace(3,1)
df['Maritime']=(df_dum.targtype1_txt_Maritime+df_dum.targtype2_txt_Maritime+df_dum.targtype3_txt_Maritime).replace(2, 1).replace(3,1)
df['Military']=(df_dum.targtype1_txt_Military+df_dum.targtype2_txt_Military+df_dum.targtype3_txt_Military).replace(2, 1).replace(3,1)
df['NGO']=(df_dum.targtype1_txt_NGO+df_dum.targtype2_txt_NGO+df_dum.targtype3_txt_NGO).replace(2, 1).replace(3,1)
df['Other']=(df_dum.targtype1_txt_Other+df_dum.targtype2_txt_Other+df_dum.targtype3_txt_Other).replace(2, 1).replace(3,1)
df['Police']=(df_dum.targtype1_txt_Police+df_dum.targtype2_txt_Police+df_dum.targtype3_txt_Police).replace(2, 1).replace(3,1)
df['Private_Citizens_Property']=(df_dum['targtype1_txt_Private Citizens & Property']+df_dum['targtype2_txt_Private Citizens & Property']+df_dum['targtype3_txt_Private Citizens & Property']).replace(2, 1).replace(3,1)
df['Religious_Figures_Institutions']=(df_dum['targtype1_txt_Religious Figures/Institutions']+df_dum['targtype2_txt_Religious Figures/Institutions']+df_dum['targtype3_txt_Religious Figures/Institutions']).replace(2, 1).replace(3,1)
df['Telecommunication']=(df_dum.targtype1_txt_Telecommunication+df_dum.targtype2_txt_Telecommunication+df_dum.targtype3_txt_Telecommunication).replace(2, 1).replace(3,1)
df['Terrorists_Non-State_Militia']=(df_dum['targtype1_txt_Terrorists/Non-State Militia']+df_dum['targtype2_txt_Terrorists/Non-State Militia']+df_dum['targtype3_txt_Terrorists/Non-State Militia']).replace(2, 1).replace(3,1)
df['Transportation']=(df_dum.targtype1_txt_Transportation+df_dum.targtype2_txt_Transportation+df_dum.targtype3_txt_Transportation).replace(2, 1).replace(3,1)
df['Utilities']=(df_dum.targtype1_txt_Utilities+df_dum.targtype2_txt_Utilities+df_dum.targtype3_txt_Utilities).replace(2, 1).replace(3,1)
df['Violent_Political_Party']=(df_dum['targtype1_txt_Violent Political Party']+df_dum['targtype2_txt_Violent Political Party']+df_dum['targtype3_txt_Violent Political Party']).replace(2, 1).replace(3,1)
df['Tourists']=(df_dum.targtype1_txt_Tourists+df_dum.targtype2_txt_Tourists+df_dum.targtype3_txt_Tourists).replace(2, 1).replace(3,1)
df['Unknown']=(df_dum.targtype1_txt_Unknown+df_dum.targtype2_txt_Unknown).replace(2, 1)

In [0]:
df.drop(columns=['targtype1_txt', 'targtype2_txt','targtype3_txt'],inplace=True)

### weapontype 1-3


See attack type for explanation.

In [0]:
#print(df.weaptype1_txt.unique())
#print(df.weaptype2_txt.unique())
#print(df.weaptype3_txt.unique())

In [0]:
df_dum=pd.get_dummies(df,columns=['weaptype1_txt','weaptype2_txt','weaptype3_txt'])

In [0]:
df['Firearms']=(df_dum.weaptype1_txt_Firearms+df_dum.weaptype2_txt_Firearms+df_dum.weaptype3_txt_Firearms).replace(2, 1).replace(3,1)
df['Explosives']=(df_dum.weaptype1_txt_Explosives+df_dum.weaptype2_txt_Explosives+df_dum.weaptype3_txt_Explosives).replace(2, 1).replace(3,1)
df['Incendiary']=(df_dum.weaptype1_txt_Incendiary+df_dum.weaptype2_txt_Incendiary+df_dum.weaptype3_txt_Incendiary).replace(2, 1).replace(3,1)
df['Chemical']=(df_dum.weaptype1_txt_Chemical+df_dum.weaptype2_txt_Chemical+df_dum.weaptype3_txt_Chemical).replace(2, 1).replace(3,1)
df['Melee']=(df_dum.weaptype1_txt_Melee+df_dum.weaptype2_txt_Melee+df_dum.weaptype3_txt_Melee).replace(2, 1).replace(3,1)
df['Sabotage_Equipment']=(df_dum['weaptype1_txt_Sabotage Equipment']+df_dum['weaptype2_txt_Sabotage Equipment']+df_dum['weaptype3_txt_Sabotage Equipment']).replace(2, 1).replace(3,1)
df['Vehicle']=(df_dum['weaptype1_txt_Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)']+df_dum['weaptype2_txt_Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)']+df_dum['weaptype3_txt_Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)']).replace(2, 1).replace(3,1)
df['Fake_Weapons']=(df_dum['weaptype1_txt_Fake Weapons']+df_dum['weaptype2_txt_Fake Weapons']+df_dum['weaptype3_txt_Fake Weapons']).replace(2, 1).replace(3,1)
df['Biological']=(df_dum.weaptype1_txt_Biological+df_dum.weaptype2_txt_Biological).replace(2, 1)
df['Radiological']=df_dum.weaptype1_txt_Radiological
df['Other']=(df_dum.weaptype1_txt_Other+df_dum.weaptype2_txt_Other+df_dum.weaptype3_txt_Other).replace(2, 1).replace(3,1)
df['Unknown']=(df_dum.weaptype1_txt_Unknown+df_dum.weaptype2_txt_Unknown+df_dum.weaptype3_txt_Unknown).replace(2, 1).replace(3,1)


In [0]:
df.drop(columns=['weaptype1_txt', 'weaptype2_txt','weaptype3_txt'],inplace=True)

### Region

In [0]:
#Turning the region into dummies in case we need it in a more wieldy manner.

#print(df.region_txt.unique())
df=pd.get_dummies(df,columns=['region_txt'])

## Duration from resolution and extended
Some of the observations in the data seem to be taking place over an extended period of time, but only have noted the start date and resolution date. instead of this. we wanted to have a single column containing the duration of the observation. 

in order to achive this, we first needed the data in the right format, and then getting an overview over it. 

In [0]:
#sets the datetime format
df['resolution'] = df['resolution'].astype('datetime64[ns]')
#creates a new column containing the duration between the start and end date
df.insert(1,'duration',(df['resolution']-df['timestamp']).dt.days)
#going of the assumption the the missing duration will just be an example of an event that did not extend beyond a day, we fill NA with 0
df['duration'].fillna(0, inplace=True)
#now we make sure that the duration data is in a moldable format for os
df['duration'] = df['duration'].astype(int)
#Now we drop the ones where the duration is under 0 since this is wrong
df = df.drop(df[(df['duration']<=0)&(df['extended']==1)].index)
# Here we loose 3101 rows, checked df.shape before and after this chunck of code

### Checking it works

In [0]:
df[df['extended']==1].head()

,timestamp,duration,extended,resolution,country_txt,latitude,longitude,success,nkill,nkillter,nwound,property,vicinity,suicide,claimed,gname,individual,crit1,crit2,crit3,Armed_Assault,Bombing_Explosion,Facility_Infrastructure_Attack,Hostage_Taking_Kidnapping,Unarmed_Assault,Hostage_Taking_Barricade_Incident,Assassination,Hijacking,Unknown,Abortion_Related,Airports_Aircraft,Business,Educational_Institution,Food_Water_Supply,Government_Diplomatic,Government_General,Journalists_Media,Maritime,Military,NGO,Other,Police,Private_Citizens_Property,Religious_Figures_Institutions,Telecommunication,Terrorists_Non-State_Militia,Transportation,Utilities,Violent_Political_Party,Tourists,Firearms,Explosives,Incendiary,Chemical,Melee,Sabotage_Equipment,Vehicle,Fake_Weapons,Biological,Radiological,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
112,1970-03-06,2,1,1970-03-08,Guatemala,14.622869,-90.529068,1,0.0,0.0,0.0,0.0,0.0,0,0.0,Rebel Armed Forces of Guatemala (FAR),0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
123,1970-03-11,4,1,1970-03-15,Brazil,-23.550711,-46.633475,1,0.0,0.0,0.0,0.0,0.0,0,0.0,Popular Revolutionary Vanguard (VPR),0,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
135,1970-03-14,1,1,1970-03-15,United States,33.766725,-118.192399,1,0.0,0.0,0.0,0.0,0.0,0,0.0,Left-Wing Militants,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
151,1970-03-24,2,1,1970-03-26,Dominican Republic,18.456792,-69.951164,1,0.0,0.0,0.0,0.0,0.0,0,0.0,Dominican Popular Movement (MPD),0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
152,1970-03-24,4,1,1970-03-28,Argentina,-27.583246,-56.683334,1,0.0,0.0,0.0,0.0,0.0,0,0.0,Argentine Liberation Front (FAL),0,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
df[df['extended']==0].head()

,timestamp,duration,extended,resolution,country_txt,latitude,longitude,success,nkill,nkillter,nwound,property,vicinity,suicide,claimed,gname,individual,crit1,crit2,crit3,Armed_Assault,Bombing_Explosion,Facility_Infrastructure_Attack,Hostage_Taking_Kidnapping,Unarmed_Assault,Hostage_Taking_Barricade_Incident,Assassination,Hijacking,Unknown,Abortion_Related,Airports_Aircraft,Business,Educational_Institution,Food_Water_Supply,Government_Diplomatic,Government_General,Journalists_Media,Maritime,Military,NGO,Other,Police,Private_Citizens_Property,Religious_Figures_Institutions,Telecommunication,Terrorists_Non-State_Militia,Transportation,Utilities,Violent_Political_Party,Tourists,Firearms,Explosives,Incendiary,Chemical,Melee,Sabotage_Equipment,Vehicle,Fake_Weapons,Biological,Radiological,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
0,1970-07-02,0,0,NaT,Dominican Republic,18.456792,-69.951164,1,1.0,0.0,0.0,0.0,0.0,0,0.0,MANO-D,0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,1970-01-01,0,0,NaT,United States,37.005105,-89.176269,1,0.0,0.0,0.0,1.0,0.0,0,0.0,Black Nationalists,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,1970-01-02,0,0,NaT,Uruguay,-34.891151,-56.187214,0,0.0,0.0,0.0,0.0,0.0,0,0.0,Tupamaros (Uruguay),0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7,1970-01-02,0,0,NaT,United States,37.791927,-122.225906,1,0.0,0.0,0.0,1.0,0.0,0,0.0,Unknown,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,1970-01-02,0,0,NaT,United States,43.076592,-89.412488,1,0.0,0.0,0.0,1.0,0.0,0,1.0,New Year's Gang,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
df.drop(columns=['extended', 'resolution'],inplace=True)

In [0]:
df.isnull().sum()

timestamp                                 0
duration                                  0
country_txt                               0
latitude                                  0
longitude                                 0
success                                   0
nkill                                     0
nkillter                                  0
nwound                                    0
property                                  0
vicinity                                  0
suicide                                   0
claimed                                   0
gname                                     0
individual                                0
crit1                                     0
crit2                                     0
crit3                                     0
Armed_Assault                             0
Bombing_Explosion                         0
Facility_Infrastructure_Attack            0
Hostage_Taking_Kidnapping                 0
Unarmed_Assault                 

## Extreme values
In some instances the data cointains values that are considered to be extreme. either because they are outliers or appear to be untrue. for example: it is highly unlikely that there exsist a terrorist attack than has taken place over 3000 days even though there is a point in the original data the would make it seem so. 

### Duration

In [0]:
#Removing Outliers in the duration column
#the upper limit is chosen, as the 95 percentile of the attacks with a duration above 1
#This was done, in order to filter out the relevant outliers
#but since most of the data has a duration of 0 these where temporarely removed in order to find the relevant number
df=df.drop(df[(df['duration']<0)|(df['duration'] >=97)].index)
df.duration.describe()

count    138075.000000
mean          0.111273
std           2.007270
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          95.000000
Name: duration, dtype: float64

### nkill and nwound

In [0]:
#This removes outliers based upon the 99 percentile.
outlier_nwound  = np.percentile(df.nwound, [0, 99])
df_out1= df[(df.nwound  > outlier_nwound[0]) & (df.nwound < outlier_nwound[1])]
outlier_nkill  = np.percentile(df.nkill, [0, 99])
#Here we contruct a new data frame where the outliers are removed in order to still have the data frame with the outliers available
df_out2 = df_out1[(df_out1.nkill  > outlier_nkill[0]) & (df_out1.nkill < outlier_nkill[1])]
df_out2.describe()

,duration,latitude,longitude,success,nkill,nkillter,nwound,property,vicinity,suicide,claimed,individual,crit1,crit2,crit3,Armed_Assault,Bombing_Explosion,Facility_Infrastructure_Attack,Hostage_Taking_Kidnapping,Unarmed_Assault,Hostage_Taking_Barricade_Incident,Assassination,Hijacking,Unknown,Abortion_Related,Airports_Aircraft,Business,Educational_Institution,Food_Water_Supply,Government_Diplomatic,Government_General,Journalists_Media,Maritime,Military,NGO,Other,Police,Private_Citizens_Property,Religious_Figures_Institutions,Telecommunication,Terrorists_Non-State_Militia,Transportation,Utilities,Violent_Political_Party,Tourists,Firearms,Explosives,Incendiary,Chemical,Melee,Sabotage_Equipment,Vehicle,Fake_Weapons,Biological,Radiological,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
count,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.0,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000,26610.00000,26610.000000,26610.000000,26610.000000,26610.000000,26610.000000
mean,0.019354,25.638468,42.691956,0.963097,3.615295,0.552574,6.656257,0.609320,0.083766,0.101090,0.164825,0.003570,0.992371,0.993123,0.833559,0.333371,0.558850,0.015708,0.007366,0.002104,0.003345,0.075085,0.001428,0.058925,0.000188,0.003194,0.078542,0.012664,0.000676,0.011800,0.085757,0.005148,0.001203,0.229651,0.003495,0.000301,0.220631,0.330365,0.023788,0.000827,0.032469,0.045284,0.003157,0.008531,0.002894,0.389929,0.601090,0.019128,0.000977,0.017512,0.000188,0.002292,0.000225,0.000113,0.0,0.000827,0.026456,0.003983,0.002706,0.022773,0.381210,0.00575,0.050620,0.322773,0.059865,0.102706,0.020331
std,0.816582,14.733248,44.853101,0.188528,3.962386,1.869236,7.757622,0.487912,0.277041,0.301453,0.371030,0.059645,0.087010,0.082644,0.372483,0.471427,0.496534,0.124347,0.085508,0.045827,0.057737,0.263533,0.037763,0.235489,0.013707,0.056429,0.269027,0.111823,0.026000,0.107987,0.280010,0.071569,0.034658,0.420616,0.059016,0.017337,0.414680,0.470353,0.152391,0.028742,0.177245,0.207930,0.056097,0.091968,0.053716,0.487743,0.489683,0.136978,0.031244,0.131172,0.013707,0.047825,0.015015,0.010617,0.0,0.028742,0.160491,0.062990,0.051947,0.149183,0.485693,0.07561,0.219225,0.467546,0.237240,0.303580,0.141132
min,0.000000,-37.818200,-157.858333,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,14.413989,35.364861,1.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

### Changeing type and z-score

In [0]:
#Here we ensure, that the dummy variables are set to be boolean values, in order to get the correct reading later.
df_out2.astype({'Armed_Assault':'bool','Bombing_Explosion':'bool','Facility_Infrastructure_Attack':'bool','Hostage_Taking_Kidnapping':'bool','Unarmed_Assault':'bool','Hostage_Taking_Barricade_Incident':'bool','Assassination':'bool','Hijacking':'bool','Unknown':'bool','Abortion_Related':'bool','Airports_Aircraft':'bool','Business':'bool','Educational_Institution':'bool','Food_Water_Supply':'bool','Government_Diplomatic':'bool','Government_General':'bool','Journalists_Media':'bool','Maritime':'bool','Military':'bool','NGO':'bool','Other':'bool','Police':'bool','Private_Citizens_Property':'bool','Religious_Figures_Institutions':'bool','Telecommunication':'bool','Terrorists_Non-State_Militia':'bool','Transportation':'bool','Utilities':'bool','Violent_Political_Party':'bool','Tourists':'bool','Unknown':'bool','Firearms':'bool','Explosives':'bool','Incendiary':'bool','Chemical':'bool','Melee':'bool','Sabotage_Equipment':'bool','Vehicle':'bool','Fake_Weapons':'bool','Biological':'bool','Radiological':'bool','Other':'bool','Unknown':'bool','property':'bool','vicinity':'bool','suicide':'bool','claimed':'bool','individual':'bool','crit1':'bool','crit2':'bool','crit3':'bool','success':'bool'}).dtypes

timestamp                                 datetime64[ns]
duration                                           int64
country_txt                                       object
latitude                                         float64
longitude                                        float64
success                                             bool
nkill                                            float64
nkillter                                         float64
nwound                                           float64
property                                            bool
vicinity                                            bool
suicide                                             bool
claimed                                             bool
gname                                             object
individual                                          bool
crit1                                               bool
crit2                                               bool
crit3                          

In [0]:
#z-score.
#in order to see where the data has the largest variance. we calculate the Z-score. where the Z-score is above 3 the data is considered to have big variance
from scipy import stats
#The Z-score is only done on the numeric columns, so we pic those out
numeric_cols = df_out2.select_dtypes(include=[np.number]).columns
zscore_df=df_out2[numeric_cols].apply(stats.zscore)
for column in zscore_df:
 max=zscore_df.loc[:,column].max()
 min=zscore_df.loc[:,column].min()
 print(column,max,min,'\n')

duration 82.02719162365287 -0.02370122881590998 

latitude 2.417131759894151 -4.307119678173532 

longitude 3.026602239050187 -4.471353241960712 

success 0.19574847450838098 -5.108596644298161 

nkill 5.3970270388002195 -0.6600426725933422 

nkillter 13.079082255983705 -0.2956206409569568 

nwound 4.556087729702511 -0.7291362252731418 

property 0.8007335972844873 -1.2488548043834817 

vicinity 3.3072781602204757 -0.3023634395279702 

suicide 2.981978086530849 -0.33534787009899597 

claimed 2.251006766704243 -0.4442456658911452 

individual 16.70644376155185 -0.05985714340363663 

crit1 0.08767756626071627 -11.405426070181008 

crit2 0.08321499904096441 -12.01706436970252 

crit3 0.446850531904166 -2.2378847704146088 

Armed_Assault 1.4140939924246783 -0.7071665712159265 

Bombing_Explosion 0.8884755163714115 -1.1255234180048779 

Facility_Infrastructure_Attack 7.915825104266751 -0.12632921860046967 

Hostage_Taking_Kidnapping 11.608846028888875 -0.08614120624147116 

Unarmed_Assault 

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


#Duplicates
Now in this we want to look through the data and make sure that there are not any duplicates laying around. 

In [0]:
#First a quick look at the number of rows present
df.shape

(138075, 70)

In [0]:
df.sort_values('nkill',ascending=False).head(5)

,timestamp,duration,country_txt,latitude,longitude,success,nkill,nkillter,nwound,property,vicinity,suicide,claimed,gname,individual,crit1,crit2,crit3,Armed_Assault,Bombing_Explosion,Facility_Infrastructure_Attack,Hostage_Taking_Kidnapping,Unarmed_Assault,Hostage_Taking_Barricade_Incident,Assassination,Hijacking,Unknown,Abortion_Related,Airports_Aircraft,Business,Educational_Institution,Food_Water_Supply,Government_Diplomatic,Government_General,Journalists_Media,Maritime,Military,NGO,Other,Police,Private_Citizens_Property,Religious_Figures_Institutions,Telecommunication,Terrorists_Non-State_Militia,Transportation,Utilities,Violent_Political_Party,Tourists,Firearms,Explosives,Incendiary,Chemical,Melee,Sabotage_Equipment,Vehicle,Fake_Weapons,Biological,Radiological,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
73126,2001-09-11,0,United States,40.697132,-73.931351,1,1384.0,5.0,8190.0,1.0,0.0,1,1.0,Al-Qaida,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
73127,2001-09-11,0,United States,40.697132,-73.931351,1,1383.0,5.0,8191.0,1.0,0.0,1,1.0,Al-Qaida,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
55934,1994-04-13,0,Rwanda,-1.932787,30.332456,1,1180.0,0.0,0.0,1.0,0.0,0,0.0,Hutu extremists,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
133225,2014-06-10,0,Iraq,36.407394,42.964626,1,670.0,0.0,0.0,1.0,0.0,0,1.0,Islamic State of Iraq and the Levant (ISIL),0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
179671,2017-10-14,0,Somalia,2.059819,45.326115,1,588.0,1.0,316.0,1.0,0.0,1,1.0,Al-Shabaab,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
#Here we check to see wheter or not there are any duplicates based upon those features. 
#theese af been selected using the 9/11 data above
df.duplicated(subset=['timestamp','country_txt','latitude','longitude','gname']).sum()

20883

In [0]:
#Here we drop the duplicates
df=df.drop_duplicates(subset=['timestamp','country_txt','latitude','longitude','gname'],keep='first')

In [0]:
#checking to see if it worked
df.sort_values('nkill',ascending=False).head(5)

,timestamp,duration,country_txt,latitude,longitude,success,nkill,nkillter,nwound,property,vicinity,suicide,claimed,gname,individual,crit1,crit2,crit3,Armed_Assault,Bombing_Explosion,Facility_Infrastructure_Attack,Hostage_Taking_Kidnapping,Unarmed_Assault,Hostage_Taking_Barricade_Incident,Assassination,Hijacking,Unknown,Abortion_Related,Airports_Aircraft,Business,Educational_Institution,Food_Water_Supply,Government_Diplomatic,Government_General,Journalists_Media,Maritime,Military,NGO,Other,Police,Private_Citizens_Property,Religious_Figures_Institutions,Telecommunication,Terrorists_Non-State_Militia,Transportation,Utilities,Violent_Political_Party,Tourists,Firearms,Explosives,Incendiary,Chemical,Melee,Sabotage_Equipment,Vehicle,Fake_Weapons,Biological,Radiological,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
73126,2001-09-11,0,United States,40.697132,-73.931351,1,1384.0,5.0,8190.0,1.0,0.0,1,1.0,Al-Qaida,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
55934,1994-04-13,0,Rwanda,-1.932787,30.332456,1,1180.0,0.0,0.0,1.0,0.0,0,0.0,Hutu extremists,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
133225,2014-06-10,0,Iraq,36.407394,42.964626,1,670.0,0.0,0.0,1.0,0.0,0,1.0,Islamic State of Iraq and the Levant (ISIL),0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
179671,2017-10-14,0,Somalia,2.059819,45.326115,1,588.0,1.0,316.0,1.0,0.0,1,1.0,Al-Shabaab,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
76347,2004-03-21,0,Nepal,27.959441,84.895897,1,518.0,500.0,216.0,1.0,0.0,0,0.0,Communist Party of Nepal- Maoist (CPN-M),0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


Now we do the same thing with the outlier data frame

In [0]:
#first check
df_out2.sort_values('nkill',ascending=False).head(5)

,timestamp,duration,country_txt,latitude,longitude,success,nkill,nkillter,nwound,property,vicinity,suicide,claimed,gname,individual,crit1,crit2,crit3,Armed_Assault,Bombing_Explosion,Facility_Infrastructure_Attack,Hostage_Taking_Kidnapping,Unarmed_Assault,Hostage_Taking_Barricade_Incident,Assassination,Hijacking,Unknown,Abortion_Related,Airports_Aircraft,Business,Educational_Institution,Food_Water_Supply,Government_Diplomatic,Government_General,Journalists_Media,Maritime,Military,NGO,Other,Police,Private_Citizens_Property,Religious_Figures_Institutions,Telecommunication,Terrorists_Non-State_Militia,Transportation,Utilities,Violent_Political_Party,Tourists,Firearms,Explosives,Incendiary,Chemical,Melee,Sabotage_Equipment,Vehicle,Fake_Weapons,Biological,Radiological,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
45754,1991-03-10,0,Mozambique,-23.022193,32.718138,1,25.0,0.0,19.0,1.0,0.0,0,0.0,Mozambique National Resistance Movement (MNR),0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
81904,2006-11-24,0,Iraq,33.303566,44.371773,1,25.0,0.0,14.0,1.0,0.0,0,0.0,Mahdi Army,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
151863,2015-08-15,0,Somalia,4.735380,45.202400,1,25.0,25.0,40.0,0.0,1.0,0,0.0,Al-Shabaab,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
27076,1986-01-12,0,El Salvador,13.967553,-89.139633,1,25.0,0.0,13.0,1.0,0.0,0,0.0,Farabundo Marti National Liberation Front (FMLN),0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
99997,2011-01-24,0,Iraq,32.604393,44.007145,1,25.0,0.0,40.0,1.0,0.0,0,0.0,Unknown,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [0]:
#Here we check to see wheter or not there are any duplicates based upon those features. 
#theese af been selected using the 9/11 data above
df_out2.duplicated(subset=['timestamp','country_txt','latitude','longitude','gname']).sum()

1826

In [0]:
#Here we drop the duplicates
df_outdup=df_out2.drop_duplicates(subset=['timestamp','country_txt','latitude','longitude','gname'],keep='first')

In [0]:
#checking to see if it worked
df_out2.sort_values('nkill',ascending=False).head(5)

,timestamp,duration,country_txt,latitude,longitude,success,nkill,nkillter,nwound,property,vicinity,suicide,claimed,gname,individual,crit1,crit2,crit3,Armed_Assault,Bombing_Explosion,Facility_Infrastructure_Attack,Hostage_Taking_Kidnapping,Unarmed_Assault,Hostage_Taking_Barricade_Incident,Assassination,Hijacking,Unknown,Abortion_Related,Airports_Aircraft,Business,Educational_Institution,Food_Water_Supply,Government_Diplomatic,Government_General,Journalists_Media,Maritime,Military,NGO,Other,Police,Private_Citizens_Property,Religious_Figures_Institutions,Telecommunication,Terrorists_Non-State_Militia,Transportation,Utilities,Violent_Political_Party,Tourists,Firearms,Explosives,Incendiary,Chemical,Melee,Sabotage_Equipment,Vehicle,Fake_Weapons,Biological,Radiological,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
45754,1991-03-10,0,Mozambique,-23.022193,32.718138,1,25.0,0.0,19.0,1.0,0.0,0,0.0,Mozambique National Resistance Movement (MNR),0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
81904,2006-11-24,0,Iraq,33.303566,44.371773,1,25.0,0.0,14.0,1.0,0.0,0,0.0,Mahdi Army,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
151863,2015-08-15,0,Somalia,4.735380,45.202400,1,25.0,25.0,40.0,0.0,1.0,0,0.0,Al-Shabaab,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
27076,1986-01-12,0,El Salvador,13.967553,-89.139633,1,25.0,0.0,13.0,1.0,0.0,0,0.0,Farabundo Marti National Liberation Front (FMLN),0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
99997,2011-01-24,0,Iraq,32.604393,44.007145,1,25.0,0.0,40.0,1.0,0.0,0,0.0,Unknown,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


# Save
lastly we save the data to a computer and transfer it to a dropbox in order to have it readily available to use in other colabs. where we perform the next steps of the analysis

In [0]:
df_out2.to_csv('clean_dataframe.csv')
from google.colab import files
files.download('clean_dataframe.csv')


df.to_csv('out_dataframe.csv')
from google.colab import files
files.download('out_dataframe.csv')
